In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import string
%matplotlib inline

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
sms = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv')

In [ ]:
sms.info()

In [ ]:
sms.describe()

In [ ]:
sms=sms.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
sms.isnull().sum()

In [ ]:
sms.rename({'v1':'label','v2':'text'}, axis =1, inplace = True)
sms.head(10)

In [ ]:
sns.countplot(sms['label'])

In [ ]:
sms['length'] = sms['text'].apply(lambda x:len(x))

In [ ]:
sns.distplot(sms['length'], kde=True)


In [ ]:
ps = PorterStemmer()
def preprocess(x):
    x = x.lower()
    x = re.sub(r'[^0-9a-zA-Z]', ' ', x)
    x = re.sub(r'\s+', ' ', x)
    l = [item for item in x if item not in string.punctuation]
    l = ''.join(l).split()
    preprocessed = [ps.stem(i) for i in l if i not in stopwords.words('english')]
    return preprocessed

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
import re
from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
sms['spam']=pd.get_dummies(sms['label'])['spam']
sms.drop(labels = 'label', axis = 1, inplace=True)

In [ ]:
X=sms['text']
y=sms['spam']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
def model(model_name, X_train, y_train, X_test, y_test):
    pipeline = Pipeline([
        ('bow',CountVectorizer(analyzer=preprocess)),
        ('tfidf', TfidfTransformer()),
        ('model', model_name)
    ])
    pipeline.fit(X_train, y_train)
    
    predictions = pipeline.predict(X_test)
    
    print (classification_report(y_test,predictions))
    print (confusion_matrix(y_test,predictions))
    print('Accuracy:', pipeline.score(X_test, y_test)*100)
    print("Training Score:",pipeline.score(X_train,y_train)*100)
    from sklearn.metrics import accuracy_score
    score = accuracy_score(y_test,predictions)
    print(score)
    return pipeline

In [ ]:
model_name=SVC()
pipeline = model(model_name,X_train,y_train,X_test,y_test)

In [ ]:
predictions = pipeline.predict(X)

In [ ]:
sms['Predicted Spam']=predictions

In [ ]:
sms.to_csv('my_submission.csv', index=False)